In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import yfinance as yf
from datetime import datetime
pd.set_option('display.max_columns', None)

In [3]:
dfscrip = pd.read_csv('/content/drive/MyDrive/ColabData/USStockScrip.csv')
print('Total script count -' , len(dfscrip))
#dfscrip.sample(3)

df3_old = pd.read_csv('/content/drive/MyDrive/ColabData/USStockClose.csv')
df3_old['Date'] = pd.to_datetime(df3_old['Date'])
print('Total record count -' , len(df3_old))
print('Max date -' , df3_old.agg({'Date':['max']}))
#df3_old.sample(3)
#df3_old.groupby('Date')['Close'].count()

Total script count - 5793
Total record count - 178720
Max date -           Date
max 2024-08-01


In [6]:
df3 = df3_old
df3['PClose'] = df3.groupby('Ticker')['Close'].shift(1)
df3['Return'] = (df3['Close'] - df3['PClose'])*100 / df3['PClose']
df3['CurrentClose'] = df3.groupby('Ticker')['Close'].transform('last')
df3['CurrentReturn'] = (df3['CurrentClose'] - df3['Close'])*100 / df3['Close']
df4 = pd.merge(df3, dfscrip[['Symbol','Security Name']], left_on='Ticker', right_on='Symbol', how='left')

#df[df['Close']>=3].sort_values(by=['Return'], ascending=True).head(10)
#df4[(df4['Close']>=3) & (df4['CurrentReturn']>=10)].sort_values(by=['Return'], ascending=True).head(3)

In [7]:
df5 = df4.sort_values(by=['Ticker', 'Date'])
df5['Return-6'] = df5.groupby('Ticker')['Close'].shift(6)
df5['Return-3'] = df5.groupby('Ticker')['Close'].shift(3)
df5['Return-2'] = df5.groupby('Ticker')['Close'].shift(2)
df5['Return-1'] = df5.groupby('Ticker')['Close'].shift(1).fillna(df5['Close'])
df5['Return+1'] = df5.groupby('Ticker')['Close'].shift(-1).fillna(df5['Close'])
df5['Return+3'] = df5.groupby('Ticker')['Close'].shift(-3).fillna(df5['Return+1'])
df5['Return+6'] = df5.groupby('Ticker')['Close'].shift(-6).fillna(df5['Return+3'])
df5['Return+12'] = df5.groupby('Ticker')['Close'].shift(-12).fillna(df5['Return+6'])
df5['Return-2'] = df5['Return-2'].fillna(df5['Return-1'])
df5['Return-3'] = df5['Return-3'].fillna(df5['Return-2'])
df5['Return-6'] = df5['Return-6'].fillna(df5['Return-3'])

#df5['Return-3'] = (df5['Close'] - df5['Return-3'])*100/df5['Close']
#df5['Return-2'] = (df5['Close'] - df5['Return-2'])*100/df5['Close']
#df5['Return-1'] = (df5['Close'] - df5['Return-1'])*100/df5['Close']
#df5['Return+1'] = (df5['Return+1'] - df5['Close'])*100/df5['Close']
#df5['Return+3'] = (df5['Return+3'] - df5['Close'])*100/df5['Close']
#df5['Return+6'] = (df5['Return+6'] - df5['Close'])*100/df5['Close']
#df5['Return+12'] = (df5['Return+12'] - df5['Close'])*100/df5['Close']

#df5[df5['Ticker']=='EPAM']

In [8]:
#data selection - select big crash stocks with some value
df6 = df5.dropna(subset=['Return'])
df7=df6.loc[df6.groupby('Ticker')['Return'].idxmin()]
df8=df7[(df7['Close']>= 5) & (df7['Return'] <= -25)].sort_values(by='Return')
print(df8.shape)
#df8.tail(3)

(953, 17)


In [9]:
# train test split
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df8, test_size=0.1, random_state=42)
print(df_train.shape)
print(df_test.shape)

(857, 17)
(96, 17)


In [10]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
scaler = StandardScaler()

X_train_raw = pd.DataFrame(df_train, columns=['Close','Return-6','Return-3','Return-2','Return-1','Return'])
X_train = scaler.fit_transform(X_train_raw[['Close','Return-6', 'Return-3','Return-2','Return-1','Return']])
y_train = pd.DataFrame()
y_train['Target'] = df_train['Return+3']

X_test_raw = pd.DataFrame(df_test, columns=['Close','Return-6','Return-3','Return-2','Return-1','Return'])
X_test_val = scaler.transform(X_test_raw[['Close','Return-6', 'Return-3','Return-2','Return-1','Return']])
y_test_val = pd.DataFrame()
y_test_val['Target'] = df_test['Return+3']

In [11]:
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

model = RandomForestRegressor()
model.fit((X_train), np.array(y_train).ravel())
y_train_pred = model.predict((X_train))
y_test_pred = model.predict(X_test_val)
model_train_r2 = r2_score(y_train, y_train_pred)
model_test_r2 = r2_score(y_test_val, y_test_pred)
print('train score {0} test score {1}'.format(model_train_r2, model_test_r2))

train score 0.9756096497861411 test score 0.6762788911061801


In [ ]:
#########################################  Prediction of new recomendations ######################################################
### read data
print(datetime.now())
dt= pd.Timestamp(datetime.now().date())
df = yf.download(dfscrip['Symbol'].tolist(), dt-pd.DateOffset(days=1) , dt)
print(datetime.now())

2024-08-29 06:14:37.679613


[*********************100%***********************]  5793 of 5793 completed
ERROR:yfinance:
4812 Failed downloads:
ERROR:yfinance:['HEWG', 'ISZE', 'WPS', 'IAUF', 'MORF', 'AGRX']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2024-08-28 00:00:00 -> 2024-08-29 00:00:00)')
ERROR:yfinance:['SNCR', 'CKPT', 'PGJ', 'NAT', 'HUN', 'HYDW', 'PAGP', 'VNO', 'XITK', 'EQH', 'EPSN', 'TYD', 'IG', 'MXF', 'RYTM', 'URI', 'CLDX', 'BC', 'MCRI', 'HQL', 'AMKR', 'PHGE', 'CET', 'TBB', 'CRNT', 'IPAC', 'JHS', 'FTNT', 'RZV', 'THQ', 'EXLS', 'NWPX', 'IWC', 'ADBE', 'AACG', 'VWO', 'RVP', 'ASUR', 'CODX', 'GFL', 'THM', 'TRTY', 'PRT', 'CTHR', 'CGW', 'VALE', 'HSDT', 'TMDX', 'TEAF', 'HLI', 'SXI', 'AA', 'MGYR', 'TRST', 'SDOG', 'NRBO', 'LYB', 'JQC', 'OGEN', 'SIGI', 'LOGC', 'VVR', 'AAOI', 'CARA', 'ADNT', 'WVE', 'CROX', 'FIZZ', 'THD', 'MU', 'ESG', 'BHLB', 'CHH', 'MGC', 'TSI', 'ROL', 'WTRG', 'DHT', 'OUSA', 'SCHR', 'OESX', 'HRB', 'OXLCP', 'BWZ', 'BLKB', 'PYN', 'RNR', 'SCVL', 'ENTR', 'XBI', 'IJR', 'F

2024-08-29 06:28:52.897912


In [ ]:
#backup latest data
df2 = df
stacked_df = df2.stack(level=1)
stacked_df2 = stacked_df.reset_index()
df3_new = stacked_df2[['Date','Ticker','Close']].sort_values(by=['Ticker','Date'])
df3_new.to_csv('/content/drive/MyDrive/ColabData/USStockCurrentmonth.csv')

In [4]:
df3_new = pd.read_csv('/content/drive/MyDrive/ColabData/USStockCurrentmonth.csv')

In [23]:
# preprocess
dt= pd.Timestamp(datetime.now().date())
df3_old = df3_old[(df3_old['Date'] < dt.replace(day=1))]

df3_all = pd.concat( [df3_old , df3_new] , axis=0 )
df3_all['PClose'] = df3_all.groupby('Ticker')['Close'].shift(1)
df3_all['Return'] = (df3_all['Close'] - df3_all['PClose'])*100 / df3_all['PClose']
df3_all['CurrentClose'] = df3_all.groupby('Ticker')['Close'].transform('last')
df3_all['CurrentReturn'] = (df3_all['CurrentClose'] - df3_all['Close'])*100 / df3_all['Close']
df4 = pd.merge(df3_all, dfscrip[['Symbol','Security Name']], left_on='Ticker', right_on='Symbol', how='left')

df5 = df4.sort_values(by=['Ticker', 'Date'])
df5['Return-6'] = df5.groupby('Ticker')['Close'].shift(6)
df5['Return-3'] = df5.groupby('Ticker')['Close'].shift(3)
df5['Return-2'] = df5.groupby('Ticker')['Close'].shift(2)
df5['Return-1'] = df5.groupby('Ticker')['Close'].shift(1).fillna(df5['Close'])
df5['Return-2'] = df5['Return-2'].fillna(df5['Return-1'])
df5['Return-3'] = df5['Return-3'].fillna(df5['Return-2'])
df5['Return-6'] = df5['Return-6'].fillna(df5['Return-3'])

#data selection - select big crash stocks with some value
df6 = df5.dropna(subset=['Return'])
df6=df6[(pd.to_datetime(df6['Date']) >= dt.replace(day=1))]
df7=df6.loc[df6.groupby('Ticker')['Return'].idxmin()]
df8=df7[(df7['Close']>= 5) & (df7['Return'] <= -25)].sort_values(by='Return')
print(df8.shape)
df_predict = df8.copy()

#scale
X_predict_raw = pd.DataFrame(df_predict, columns=['Close','Return-6','Return-3','Return-2','Return-1','Return'])
X_predict = scaler.transform(X_predict_raw[['Close','Return-6', 'Return-3','Return-2','Return-1','Return']])

#run model
y_test_pred = model.predict(X_predict)

TypeError: Invalid comparison between dtype=datetime64[ns, UTC] and Timestamp

In [24]:
#display
y_pred_df = pd.DataFrame(y_test_pred, columns=['Pred_Target'])
df_predict_col = df_predict[['Symbol','Return','Date','Return-6','Return-3','Return-2','Return-1', 'Close','CurrentClose','Security Name']]

y_test_verify = y_pred_df
y_test_verify2 = pd.concat([y_test_verify , df_predict_col.reset_index() ], axis=1)
y_test_verify2['Pred Target Return%'] = (y_test_verify2['Pred_Target'] - y_test_verify2['Close'])*100/y_test_verify2['Close']
y_test_verify2 = y_test_verify2.drop(columns=['Close'])
X_test_verify = y_test_verify2

#y_test_verify2['outcome'] = np.where(y_test_verify2['Pred Target Return%'] > 10, np.where(y_test_verify2['Act Target Return%'] > 10, 'Buy - Profit', 'Buy - Loss') , 'Ignore')
strategy_conditions = [
    (X_test_verify['Pred Target Return%'] <= 10),
    (X_test_verify['Pred Target Return%'] > 10) & (X_test_verify['Return-6'] > X_test_verify['Return-1']) ,
    (X_test_verify['Pred Target Return%'] > 10)
]
strategy_choices = ['Ignore','Risk Buy','Buy']
X_test_verify['strategy'] = np.select(strategy_conditions, strategy_choices, default='unknown')

#X_test_verify[X_test_verify['Target'] != 0].sort_values('diff', ascending=False).head(3)
#X_test_verify[X_test_verify['Date'] == pd.to_datetime('2023-07-03')].sort_values('Pred Target Return%', ascending=False).head(10)
#X_test_verify.sample(1)
recomend = X_test_verify[['Symbol','Security Name' , 'Date',  'Return', 'Return-6',
       'Return-3', 'Return-2', 'Return-1', 'CurrentClose' ,'Pred_Target','Pred Target Return%', 'strategy']]

In [ ]:
recomend[(recomend['strategy'] == 'Buy')].sort_values('CurrentClose', ascending=False)

In [29]:
start_date = '2022-04-01'
end_date = datetime.now()
df_detail = yf.download("MRNA", start_date, end_date)
#df_detail.reset_index(inplace=True)
#df_detail['Date']=pd.to_datetime(df_detail['Date'])
sns.lineplot(df_detail, x='Date', y='Close')
plt.xticks(rotation=45)
plt.show()

In [25]:
X_test_verify.groupby(['Date','strategy'])['strategy'].count()

In [26]:
recomend[(recomend['strategy'] == 'Ignore')].sort_values('CurrentClose', ascending=False).head(30)